In [1]:
%env THEANO_FLAGS="device=gpu2"

env: THEANO_FLAGS="device=gpu2"


In [2]:
import theano
import theano.tensor as T
theano.config.floatX='float32'

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 2: Tesla K80 (CNMeM is enabled with initial size: 22.0% of memory, cuDNN not available)


In [4]:
from lasagne.layers import *
import lasagne
from hierarchical_softmax import HierarchicalSoftmaxDenseLayer

In [5]:
l_in = InputLayer([100,256])
target_y = T.ivector()

n_outputs = 10**6

l_out_softmax = DenseLayer(l_in, n_outputs, nonlinearity=lasagne.nonlinearities.softmax)
l_out_hsoftmax = HierarchicalSoftmaxDenseLayer(l_in, n_outputs, target=target_y)

In [6]:
import numpy as np
data = [(np.random.normal(size=[100,256]).astype('float32'),
         np.random.randint(0,n_outputs,size=[100],dtype='int32')) for i in range(100)]


In [7]:
from lasagne.objectives import categorical_crossentropy as loss
f_softmax = theano.function([l_in.input_var, target_y], loss(get_output(l_out_softmax),target_y).mean())
f_hsoftmax = theano.function([l_in.input_var, target_y], get_output(l_out_hsoftmax).mean())

In [10]:
%%time
#volatile gpu util 80~100%, titanx, NO CNMEM!
for batch in data:
    f_softmax(*batch)

CPU times: user 4.49 s, sys: 1.31 s, total: 5.8 s
Wall time: 5.98 s


In [11]:
%%time
#volatile gpu util 90~100%, titanx, NO CNMEM!
for batch in data:
    f_hsoftmax(*batch)

CPU times: user 136 ms, sys: 24 ms, total: 160 ms
Wall time: 187 ms
